# Project Notebook
Generated from script.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to sys.path
# Assuming notebook is in 'notebooks/' and project root is one level up
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
from pokemon_predictor import config
from pokemon_predictor.tabular import load_data
import pandas as pd
import numpy as np
import joblib


# Transfer Learning with MobileNetV2

In [ ]:
from pokemon_predictor.images import get_pokemon_dataset
from pokemon_predictor import config
import tensorflow as tf
from tensorflow.keras import applications, layers, models, optimizers

train_gen, val_gen, classes = get_pokemon_dataset(batch_size=32, target_size=config.IMG_SIZE)

base = applications.MobileNetV2(input_shape=(*config.IMG_SIZE, 3), include_top=False, weights='imagenet')
base.trainable = False

inputs = layers.Input(shape=(*config.IMG_SIZE, 3))
x = layers.Rescaling(scale=1./127.5, offset=-1)(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(len(classes), activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=10)
